# CNN to solve a multiband linear equation system

In [8]:
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader

# from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

## Samples Generator

In [68]:
def tensor2matrix(t):

    pass

def make_matrix(number_generator,
                args={},
                symmetric=True,
                off_diagonal_abs_mean=0.5,
                grid_size=(5, 5),
                as_tensor=False):
    ni, nj = grid_size
    n_diag = ni*nj
    n_diag_1 = ni*nj - 1
    n_diag_ni = ni*(nj-1)

    diag = [1.]*n_diag
    diag_1_u = [number_generator(**args) for _ in range(n_diag_1)]
    diag_ni_u = [number_generator(**args) for _ in range(n_diag_ni)]

    if symmetric:
        diag_1_l = diag_1_u
        diag_ni_l = diag_ni_u
    else:
        diag_1_l = [number_generator(**args) for _ in range(n_diag_1)]
        diag_ni_l = [number_generator(**args) for _ in range(n_diag_ni)]

    diag_1_u[ni-1::ni] = [0]*(nj-1)
    diag_1_l[ni-1::ni] = [0]*(nj-1)

    diag = torch.tensor(diag).float()
    diag_1_u = torch.tensor(diag_1_u).float()
    diag_1_l = torch.tensor(diag_1_l).float()
    diag_ni_u = torch.tensor(diag_ni_u).float()
    diag_ni_l = torch.tensor(diag_ni_l).float()

    # t = torch.cat([diag_ni_u.reshape((ni,nj)), diag_1_u, diag_1_l, diag_ni_l])


    m = torch.zeros((n_diag,n_diag))
    m = m + torch.diag_embed(diag)
    m[1:,:-1] = m[1:,:-1] + torch.diag_embed(diag_1_l)
    m[:-1,1:] = m[:-1,1:] + torch.diag_embed(diag_1_u)
    m[ni:,:-ni] = m[ni:,:-ni] + torch.diag_embed(diag_ni_l)
    m[:-ni,ni:] = m[:-ni,ni:] + torch.diag_embed(diag_ni_u)

    return m

def one():
    return 1

m = make_matrix(#number_generator=random.uniform,
                #args={'a':1,'b':3},
                one,
                grid_size=(4,2))
m

tensor([[1., 1., 0., 0., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 1., 0., 0.],
        [0., 1., 1., 1., 0., 0., 1., 0.],
        [0., 0., 1., 1., 0., 0., 0., 1.],
        [1., 0., 0., 0., 1., 1., 0., 0.],
        [0., 1., 0., 0., 1., 1., 1., 0.],
        [0., 0., 1., 0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 0., 0., 1., 1.]])